In [1]:
import pickle
import numpy as np
import json
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')


In [10]:
with open('Files/titles.pkl', 'rb') as f:
    titles = pickle.load(f)

with open('Files/embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

with open('Files/authors.pkl', 'rb') as f:
    authors = pickle.load(f)

with open('Files/years.pkl', 'rb') as f:
    years = pickle.load(f)

with open('Files/summary.pkl', 'rb') as f:
    summary = pickle.load(f)

index = pickle.load(open('Files/index.sav' , 'rb'))

#model = pickle.load(open('Files/model.sav' , 'rb'))
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

In [3]:
# simple query to return the first 5 similar documents (Titles, Authors, Years, Summary)

def retrieve(query, k=5):
    xq = model.encode([query])
    D, I = index.search(xq, k)

    results = []
    for i in range(k):
        results.append(
            { 
                'Title': titles[I[0][i]],
                'Author': authors[I[0][i]],
                'Year': years[I[0][i]],
                'Summary': summary[I[0][i]]
            }
        )

    return results

In [8]:
search = 'What is the best way to learn Python?'
xq = model.encode([search])
D, I = index.search(xq, 5)

print(I)
print(D)

[[ 7128 26927 11648  4854  9998]]
[[0.9771748 0.9900106 1.0068038 1.0124968 1.0243872]]


In [15]:
res_titles = [ titles[I[0][i]] for i in range(5) ]
res_authors = [ authors[I[0][i]] for i in range(5) ]
res_years = [ years[I[0][i]] for i in range(5) ]
res_summary = [ summary[I[0][i]] for i in range(5) ]



print(res_titles)
print(res_authors)
print(res_years)
print(res_summary)

['Minecraft: An Engaging Platform to Learn Programming', 'Improving Learning from Demonstrations by Learning from Experience', 'Pen and Paper Exercises in Machine Learning', 'The Physics of Learning', 'Discover Life Skills for Planning with Bandits via Observing and\n  Learning How the World Works']
['Worasait Suwannik', 'Marcelo H Ang Jr', 'Michael U. Gutmann', 'Sahar Basiri-Esfahani', 'Tin Lai']
[2022, 2021, 2022, 2022, 2022]
['Teaching programming effectively is difficult. This paper explores the\nbenefits of using Minecraft Education Edition to teach Python programming.\nEducators can use the game to teach various programming concepts ranging from\nfundamental programming concepts, object-oriented programming, event-driven\nprogramming, and parallel programming. It has several benefits, including being\nhighly engaging, sharpen creativity and problem-solving skill, motivating the\nstudy of mathematics, and making students realizes the importance of\nprogramming.', 'How to make imit

----

In [5]:
import gradio as gr

iface = gr.Interface(
    fn=retrieve,
    inputs = gr.inputs.Textbox(lines=1, placeholder="Enter Query...", label="Query text"),
    outputs = [
        gr.outputs.Textbox(label="Title"),
        gr.outputs.Textbox(label="Author"),
        gr.outputs.Textbox(label="Year"),
        gr.outputs.Textbox(label="Summary")
    ]
)

iface.launch(inline=True, share=False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fdea26c2ca0>, 'http://127.0.0.1:7860/', None)